In [17]:
#Import necessary packages
import pandas as pd
import numpy as np
import math as math
import neighborhood as nbr
import seaborn as sns
import matplotlib.pyplot as plt
import time as time
import torch as torch
import warnings
warnings.filterwarnings('ignore')

Below is an import function written with some assistance from Josh Gorin to allow for an additional input into the neighborhood optimizer function

In [18]:
class Dataset(pd.DataFrame):
    def __init__(self, data:pd.DataFrame):
        super().__init__(data=data, index=data.index, columns=data.columns)
        assert self["TC"] is not None, "given dataset does not contain TC parameter."
        assert self["thr"] is not None, "given dataset does not contain thr parameter."
        assert self["ln(D/a^2)"] is not None, "given dataset does not contain ln(D/a^2) parameter."
        assert self["Fi"] is not None, "given dataset does not contain Fi parameter."

        self.np_TC = torch.tensor(self["TC"].values) 
        self.np_thr = torch.tensor(self["thr"].values) 
        self.np_lnDaa = torch.tensor(self["ln(D/a^2)"].values) 
        self.np_Fi_exp = torch.tensor(self["Fi"].values) 


In [19]:
class Objective:
    def __init__(self, fn, data:Dataset):
        self.fn = fn
        self.dataset = data

    def evaluate(self, X):
        return self.fn(X, self.dataset)

In [20]:
# Code written by Josh Gorin to allow for an additional input into the neighborhood optimizer function
import neighborhood as nbr
import pandas as pd


class Optimizer(nbr.Searcher):
    
    def __init__(self, dataset_path:str, objective_fn, limits:list[float], num_samp:int, 
                 num_resamp:int, names:list[str], maximize:bool=False, verbose:bool=True):
        
        if names == None:
            names = []
            
        super().__init__(
            objective_fn, 
            limits, 
            num_samp, 
            num_resamp, 
            names=names, 
            maximize=maximize, 
            verbose=verbose
        )
        
        self._objective = Objective(
            objective_fn, 
            Dataset(pd.read_csv(dataset_path))
        
        )
        
    def update(self, num_iter=10):
        """
        tweaked from original codebase to pass in training data into objective function
        """

        for ii in range(num_iter):
            
            # If the first iteration, or every 10th... take a random sample
            if not self._sample or ii % 10 == 0:
                self._random_sample()
            else:
                self._neighborhood_sample()
                        
            # execute forward model for all samples in queue
            while self._queue:
                param = self._queue.pop()
                result = self._objective.evaluate(param)
                self._sample.append({
                    'param': param,
                    'result': result,
                    'iter': self._iter
                    })
             
            # prepare for next iteration
            self._sample.sort(key=lambda x: x['result'], reverse=self._maximize)
            self._iter += 1
            if self._verbose:
                print(self)

The below will be a cell for calculating D0 and D/a^2 from experimental data. This code is adapted from Marissa Tremblay's 2014 implementation of the Fechtig and Kalbitzer equations

In [21]:
def D0calc_MonteCarloErrors(expdata):
# Function for calculating D0 and D/a^2 from experimental data. Input should be a
# Pandas DataFrame with columns "TC", "thr",
# M, and, and delM, which correspond to heating temperature (deg C), 
# heating step duration (time in hours),
# M (measured concentration in cps, atoms, or moles), delM (same units)
    
    # Calculate diffusivities from the previous experiment
    TC = expdata.loc[:,"TC"].array
    thr = expdata.loc[:,"thr"].array
    M = expdata.loc[:,"M"].array
    delM = expdata.loc[:,"delM"].array

    #Check if units are in minutes and convert from hours to minutes if necesssary
    if thr[1]>4:
        thr = thr/60

    #Convert units
    TK = 273.15+TC
    tsec = thr*60*60
    Tplot = 1*10**4/TK
    nstep = len(M)
    cumtsec = np.cumsum(tsec)
    Si = np.cumsum(M)
    S = np.amax(Si)
    Fi = Si/S


    # initialize diffusivity vectors fore each Fechtig and Kalbitzer equation
    DR2_a = np.zeros([nstep])
    DR2_b = np.zeros([nstep])
    DR2_c = np.zeros([nstep])

    # Create the a list of times for each heating step
    diffti = cumtsec[1:]-cumtsec[0:-1]

    # Create a list of the gas fraction differences between steps
    diffFi = Fi[1:]-Fi[0:-1]


    # use equations 5a through c from Fechtig and Kalbitzer for spherical geometry
    # Fechtig and Kalbitzer Equation 5a, for cumulative gas fractions up to 10%
    # special case when i = 1; need to insert 0 for previous amount released

    DR2_a[0] = ( (Fi[0]**2 - 0.**2 )*math.pi/(36*(cumtsec[0])))


    # Equation 5a for all other steps

    DR2_a[1:] = ((Fi[1:])**2 - (Fi[0:-1])**2 )*math.pi/(36*(diffti))

    # Fechtig and Kalbitzer Equation 5b, for cumulative gas fractions between 10 and 90%

    DR2_b[0] = (1/((math.pi**2)*tsec[0]))*((2*math.pi)-((math.pi*math.pi/3)*Fi[0])\
                                           - (2*math.pi)*(np.sqrt(1-(math.pi/3)*Fi[0])))
    DR2_b[1:] = (1/((math.pi**2)*diffti))*(-(math.pi*math.pi/3)*diffFi \
                                           - (2*math.pi)*( np.sqrt(1-(math.pi/3)*Fi[1:]) \
                                            - np.sqrt(1 - (math.pi/3)*Fi[0:-1]) ))

    # Fechtig and Kalbitzer Equation 5c, for cumulative gas fractions greater than 90%
    DR2_c[1:] = (1/(math.pi*math.pi*diffti))*(np.log((1-Fi[0:-1])/(1-Fi[1:])))

    # Decide which equation to use based on the cumulative gas fractions from each step
    use_a = (Fi<= 0.1) & (Fi> 0.00000001)
    use_b = (Fi > 0.1) & (Fi<= 0.85)
    use_c = (Fi > 0.85) & (Fi<= 1.0)

    # Compute the final values
    DR2 = use_a*DR2_a + np.nan_to_num(use_b*DR2_b) + use_c*DR2_c

    # Compute uncertainties in diffusivity using a Monte Carlo simulation
    # Generates simulated step degassing datasets, such that each step of the 
    # experiment has a Gaussian distribution centered at M and with 1s.d. of
    # delM across the simulated datasets.Then recomputes diffusivities for each 
    # simulated dataset and uses the range of diffusivities for each step across
    # all simulated datasets to estimate uncertainty. 
    # make vector with correct diffusivites for each step

    n_sim = 30000 #number of simulations in the monte carlo
    MCsim = np.zeros([nstep,n_sim])#initialize matrix for simulated measurements


    
    
    for i in range(nstep):
        #Generate the simulated measurements
        MCsim[i,:] = np.random.randn(1,n_sim)*delM[i] + M[i]

    #compute cumulative gas release fraction for each simulation
    MCSi = np.cumsum(MCsim,0)
    MCS = np.amax(MCSi,0)
    MCFi = np.zeros([nstep,n_sim])
    delMCFi = np.zeros([nstep,1])
    MCFimean = np.zeros([nstep,1])



    for i in range(n_sim):
        MCFi[:,i] = MCSi[:,i]/np.amax(MCSi[:,i])
    for i in range(nstep):
        delMCFi[i] = (np.amax(MCFi[i,:],0) - np.amin(MCFi[i,:],0))/2
        MCFimean[i] = np.mean(MCFi[i,:],0)
      
    #Initialize vectors
    MCDR2_a = np.zeros([nstep,n_sim])
    MCDR2_b = np.zeros([nstep,n_sim])
    MCDR2_c = np.zeros([nstep,n_sim])
    MCdiffFi = np.zeros([nstep,n_sim])



    for m in range(1,nstep): #For step of each experiment...
        for n in range(n_sim):
            MCdiffFi[m,n] = MCFi[m,n] - MCFi[m-1,n] #calculate the fraction released at each step
    for n in range(n_sim): #For each first step of an experiment, insert 0 for previous amount released
        MCDR2_a[0,n] = ((MCFi[m,n])**2 - (MCFi[m-1,n])**2 )*math.pi/(36*(diffti[m-1]))
    for m in range(1,nstep): #Calculate fechtig and kalbitzer equations for each fraction
        for n in range(n_sim):
            MCDR2_a[m,n] = ( (MCFi[m,n])**2 - (MCFi[m-1,n])**2 )*math.pi/(36*(diffti[m-1]));
            MCDR2_b[m,n] = (1/((math.pi**2)*diffti[m-1]))*( -(math.pi*math.pi/3)* MCdiffFi[m,n] \
                            - (2*math.pi)*( np.sqrt(1-(math.pi/3)*MCFi[m,n]) \
                            -np.sqrt(1 - (math.pi/3)*MCFi[m-1,n]) ))
            MCDR2_c[m,n] = (1/(math.pi*math.pi*diffti[m-1]))*(np.log((1-MCFi[m-1,n])/(1-MCFi[m,n])));

    use_a_MC = (MCFi<= 0.1) & (MCFi> 0.00000001)
    use_b_MC = (MCFi > 0.1) & (MCFi<= 0.85)
    use_c_MC = (MCFi > 0.85) & (MCFi<= 1.0) 


    MCDR2 = use_a_MC*MCDR2_a + np.nan_to_num(use_b_MC*MCDR2_b) + use_c_MC*MCDR2_c

    MCDR2_uncert = np.zeros([nstep,1])
    for i in range(nstep):
        MCDR2_uncert[i,0] = np.std(MCDR2[i,:]) 


    return pd.DataFrame({"Tplot": Tplot,"Fi": MCFimean.ravel(),"Fi uncertainty": \
                               delMCFi.ravel(), "Daa": DR2,"Daa uncertainty": MCDR2_uncert.ravel(), \
                               "ln(D/a^2)": np.log(DR2),"ln(D/a^2)-del": np.log(DR2-MCDR2_uncert.ravel()), \
                               "ln(D/a^2)+del": np.log(DR2+MCDR2_uncert.ravel()) })

The following code is for use with the neighborhood algorithm as an objective function. It takes in diffusion kinetics (X) and experimental data (data) and returns a misfit between the two.

In [22]:
def forwardModelKinetics(kinetics,expData): 
    # kinetics: (Ea, lnd0aa_x, fracs_x). To make this compatible with other functions, if there are x fracs, input x-1 fractions, and the code will determine the
    # final fraction.
    
    R = 0.008314 #gas constant
    torch.pi = torch.acos(torch.zeros(1)).item() * 2
     # Parameters that need to be read in (These I'll likely read from a file eventually
    # But we'll read the data from above just to test the function for now...)
    if len(kinetics) <= 3:
        ndom = 1
    else:
        ndom = (len(kinetics))//2

    # Make a subset of X, removing the Ea so that we have an even number of elements
    temp = kinetics[1:]
 
    if type(expData) is tuple:
        TC = expData[0]
        thr = expData[1]
        lnDaa = expData[2]
        Fi = expData[3]

    else:
        TC = expData.np_TC
        thr = expData.np_thr
        lnDaa = expData.np_lnDaa
        Fi = expData.np_Fi_exp
    
    # Grab the parameters from the input
    lnD0aa = torch.tile(temp[0:ndom],(len(thr),1)) #lnD0aa = np.tile(lnD0aa,(len(thr),1))
    fracstemp = temp[ndom:]

    fracs = torch.tile(torch.concat((fracstemp,1-torch.sum(fracstemp,axis=0,keepdim=True)),axis=-1),(len(thr),1))
    Ea = torch.tile(kinetics[0],(len(thr),ndom)) # This is an Ea for each domain

    # Put variables in correct units

    tsec = torch.tile(torch.reshape(thr*3600,(-1,1)),(1,Ea.shape[1])) #This is a complicated way of getting tsec into a numdom x numstep matrix for multiplication
    cumtsec = torch.tile(torch.reshape(torch.cumsum(thr*3600,dim=0),(-1,1)),(1,Ea.shape[1])) #Same as above, but for cumtsec                                                         
    TK = torch.tile(torch.reshape((TC + 273.15),(-1,1)),(1,Ea.shape[1])) #This is a complicated way of turning TC from a 1-d array to a 2d array and making two column copies of it
    

    Daa = torch.exp(lnD0aa)*torch.exp(-Ea/(R*TK))
    
    # Pre-allocate fraction and Dtaa
    f = torch.zeros(Daa.shape)
    Dtaa = torch.zeros(Daa.shape)
    DaaForSum = torch.zeros(Daa.shape)
    
    
    #Calculate D
    DaaForSum[0,:] = Daa[0,:]*tsec[0,:]
    DaaForSum[1:,:] = Daa[1:,:]*(cumtsec[1:,:]-cumtsec[0:-1,:])

    Dtaa = torch.cumsum(DaaForSum, axis = 0)

    f = (6/(math.pi**(3/2)))*torch.sqrt((math.pi**2)*Dtaa)
    f[f>=0.1] = (6/(torch.pi**(3/2)))*torch.sqrt((torch.pi**2)*Dtaa[f>=0.1])-(3/(torch.pi**2))* \
            ((torch.pi**2)*Dtaa[f>=0.1])
    f[f>=0.9] = 1 - (6/(torch.pi**2))*torch.exp(-(torch.pi**2)*Dtaa[f>=0.9])

    # If any member of f is preceeded by a value greater than it, set equal to 1 (we've reached total gas released)
    f[1:,:][f[1:,:]<f[0:-1,:]] = 1
    f[1:,:][f[0:-1,:]==1] = 1

    # Multiply each gas realease by the percent gas located in each domain
    f_MDD = f*fracs

    # Calculate the total gas released at each step from each gas fraction
    sumf_MDD = torch.sum(f_MDD,axis=1)
    

    #Calculate the apparent Daa from the MDD using equations of Fechtig and Kalbitzer
    Daa_MDD_a = torch.zeros(sumf_MDD.shape)
    Daa_MDD_b = torch.zeros(sumf_MDD.shape)
    Daa_MDD_c = torch.zeros(sumf_MDD.shape)
   
    # use equations 5a through c from Fechtig and Kalbitzer for spherical geometry
    # Fechtig and Kalbitzer Equation 5a, for cumulative gas fractions up to 10%
    # special case when i = 1; need to insert 0 for previous amount released

    # Rewrite the cumtsec as just one column to make for easier calculations below 
    # and because we don't need to use on three domains separately anymore
    cumtsec = cumtsec[:,0]
    diffti = cumtsec[1:]-cumtsec[0:-1]
    diffFi = sumf_MDD[1:]-sumf_MDD[0:-1]

    Daa_MDD_a[0] = ( (sumf_MDD[0]**2 - 0.**2 )*torch.pi/(36*(cumtsec[0])))


    # Equation 5a for all other steps

    Daa_MDD_a[1:] = ((sumf_MDD[1:])**2 - (sumf_MDD[0:-1])**2 )*math.pi/(36*(diffti))

    # Fechtig and Kalbitzer Equation 5b, for cumulative gas fractions between 10 and 90%
    Daa_MDD_b[0] = (1/((torch.pi**2)*tsec[0,0]))*((2*torch.pi)-((math.pi*math.pi/3)*sumf_MDD[0])\
                                           - (2*math.pi)*(torch.sqrt(1-(math.pi/3)*sumf_MDD[0])))
    Daa_MDD_b[1:] = (1/((math.pi**2)*diffti))*(-(math.pi*math.pi/3)*diffFi \
                                           - (2*math.pi)*( torch.sqrt(1-(math.pi/3)*sumf_MDD[1:]) \
                                            - torch.sqrt(1 - (math.pi/3)*sumf_MDD[0:-1]) ))

    # Fechtig and Kalbitzer Equation 5c, for cumulative gas fractions greater than 90%
    Daa_MDD_c[1:] = (1/(math.pi*math.pi*diffti))*(torch.log((1-sumf_MDD[0:-1])/(1-sumf_MDD[1:])))

    # Decide which equation to use based on the cumulative gas fractions from each step
    use_a = (sumf_MDD<= 0.1) & (sumf_MDD> 0.00000001)
    use_b = (sumf_MDD > 0.1) & (sumf_MDD<= 0.85)
    use_c = (sumf_MDD > 0.85) & (sumf_MDD<= 1.0)
    Daa_MDD = use_a*Daa_MDD_a + torch.nan_to_num(use_b*Daa_MDD_b) + use_c*Daa_MDD_c
    

    lnDaa_MDD = torch.log(Daa_MDD)
    
    return (TC,lnDaa,sumf_MDD)
    #return pd.DataFrame({"TC": TC.ravel(),"ln(D/a^2)_MDD": lnDaa_MDD.ravel(),"Fi_MDD": sumf_MDD.ravel()}) # Return a dataframe with the results forward modeled   

In [23]:
def objectiveFunction(X,data):
    torch.pi = torch.acos(torch.zeros(1)).item() * 2
    # Below here will eventually get turned into a function
    # Code written by Marissa Tremblay and modified/transcribed into Python by Drew Gorin.
    #Last modified 1.2023.

    # This function calculates the fraction of gas released from each domain
    # in an MDD model during the heating schedule used in the diffusion
    # experiment. Then the fractions released from each domain are combined in
    # proportion to one another as specified by the MDD model, and the
    # diffusivity of each step is calculated. A residual is calculated as the
    # sum of absolute differences between the observed and modeled release
    # fractions over all steps.
    
    #Time both of these
    #X = torch.from_numpy(X)
    X = torch.as_tensor(X)
    # Unpack the parameters and spit out a high misfit value if constraints are violated
    if len(X) <= 3:
        ndom = 1
    else:
        ndom = (len(X))//2


    # Grab the other parameters from the input
    
    temp = X[1:]
    lnD0aa = temp[0:ndom]
    fracstemp = temp[ndom:] 
    sumTemp = (1-torch.sum(fracstemp,axis=0,keepdim = True))
    fracs = torch.concat((fracstemp,sumTemp),dim=-1)
    # Report high misfit values if conditions are not met

    
    for i in range(len(lnD0aa)-1):
        if lnD0aa[i+1]>lnD0aa[i]:
            return 10**10
        
    fwdModelResults = forwardModelKinetics(X,data)

    # Parameters that need to be read in (These I'll likely read from a file eventually
    # But we'll read the data from above just to test the function for now...)

    TC = data.np_TC #data["TC"].to_numpy()
    thr = data.np_thr#["thr"].to_numpy()
    lnDaa = data.np_lnDaa#["ln(D/a^2)"].to_numpy()
    Fi_exp = data.np_Fi_exp#["Fi"].to_numpy()
    Fi_MDD = fwdModelResults[2]#fwdModelResults["Fi_MDD"].to_numpy()

    
    #Calculate the Fraction released for each heating step
    TrueFracFi = (Fi_exp[1:]-Fi_exp[0:-1])
    TrueFracFi = torch.concat((torch.unsqueeze(Fi_exp[0],dim=-1),TrueFracFi),dim=-1)
    

    trueFracMDD = Fi_MDD[1:]-Fi_MDD[0:-1]
    trueFracMDD = torch.concat((torch.unsqueeze(Fi_MDD[0],dim=-1),trueFracMDD),dim=-1)
    # Calculate the L1 loss 
    misfit = torch.absolute(TrueFracFi-trueFracMDD)

    # Assign penalty for each step if the model runs out of gas before the experiment did

    # Add a misfit penalty of 1 for each heating step that ran out of gas early
    misfit[trueFracMDD==0] = 1
    
    if torch.round(Fi_MDD[-1],decimals=2) != 1:
         return 10**10
    

#     if torch.round(Fi_MDD[-2],decimals=2) >= 1:
#          return 10**10
   # Return the sum of the residuals
    return torch.sum(misfit)

In [34]:
# I wrote this to accomodate a python optimizer at one point, but this is now out of date. don't use!

def objectiveFunction4PythonOptimizer(X,*data):

    # *data is a tuple of the data from the experiment in the order ("TC", "thr", "ln(D/a^2)", "Fi")
    
     # Below here will eventually get turned into a function
    # Code written by Marissa Tremblay and modified/transcribed into Python by Drew Gorin.
    #Last modified 1.2023.

    # This function calculates the fraction of gas released from each domain
    # in an MDD model during the heating schedule used in the diffusion
    # experiment. Then the fractions released from each domain are combined in
    # proportion to one another as specified by the MDD model, and the
    # diffusivity of each step is calculated. A residual is calculated as the
    # sum of absolute differences between the observed and modeled release
    # fractions over all steps.

    
    # Unpack the parameters and spit out a high misfit value if constraints are violated
    if len(X) <= 3:
        ndom = 1
    else:
        ndom = (len(X))//2


    # Grab the other parameters from the input
    
    temp = X[1:]
    lnD0aa = temp[0:ndom]
    fracstemp = temp[ndom:]
    if torch.sum(fracstemp) >=1:
        return 10**10
    
    fracs = torch.append(fracstemp,1-torch.sum(fracstemp))
    
    # Report high misfit values if conditions are not met
    if torch.sum(fracs)!=1:
        return 10**10
    
    for i in range(len(lnD0aa)-1):
        if lnD0aa[i+1]>lnD0aa[i]:
            return 10**10
        
        
    fwdModelResults = forwardModelKinetics(X,data)

    # Parameters that need to be read in (These I'll likely read from a file eventually
    # But we'll read the data from above just to test the function for now...)
    
    TC = data[0]
    thr = data[1]
    lnDaa = data[2]
    Fi_exp = data[3]
    Fi_MDD = fwdModelResults["Fi_MDD"].to_numpy()

    #Calculate the Fraction released for each heating step
    trueFracFi = (Fi_exp[1:]-Fi_exp[0:-1])
    trueFracFi = torch.insert(trueFracFi,0,Fi_exp[0])

    trueFracMDD = Fi_MDD[1:]-Fi_MDD[0:-1]
    trueFracMDD = torch.insert(trueFracMDD,0,Fi_MDD[0])

    # Calculate the L1 loss 
    misfit = torch.absolute(trueFracFi-trueFracMDD)

    # Assign penalty for each step if the model runs out of gas before the experiment did

    # Add a misfit penalty of 1 for each heating step that ran out of gas early
    misfit[trueFracMDD==0] = 1

    # Return the sum of the residuals
    if Fi_MDD[-1] != 1:
        return 10**10
        
    return torch.sum(misfit)

In [25]:
def plotModelResults(fwdModel,expData):
    # Calculate the temp in 10000/TK (Standard plotting units for this field)
    expData["10000/TK"] = 10000/(expData["TC"]+273.15)
    fwdModel["10000/TK"] = 10000/(fwdModel["TC"]+273.15)
    fwdModel["MDDFi_Cum"] = np.cumsum(fwdModel["Fi_MDD"])
    expData["Fi_Cum"] = np.cumsum(expData["Fi"])
    
    
    plt.figure();
    plt.subplot(1,2,1)
    sns.scatterplot(data = expData, x = "10000/TK", y = "ln(D/a^2)")
    sns.scatterplot(data = fwdModel, x = "10000/TK",y="ln(D/a^2)_MDD")
    plt.legend(["Experiment","Model"])
    plt.subplot(1,2,2)
    sns.scatterplot(data = expData, x =expData.index, y = "Fi")
    sns.scatterplot(data = fwdModel, x =fwdModel.index, y="Fi_MDD")
    plt.legend(["Experiment","Model"])


In [26]:
# This is the MAIN function

#Code written by Marissa Tremblay and transcribed into Python/modified by Drew Gorin. Last modified 1.2023

#This m-file is used to fit an MDD model to stepwise degassing diffusion
#experiment data. It is currently set up for only one isotope. The number
#of domains is allowed to vary. The activation energy is assumed to be the
#same across domains while the pre-exponential factor (D0/a^2) and the
#fraction of gas in each domain varies. Needs the companion functions
#D0calc_MonteCarloErros.m and TremblayMDD.m.

# USER INPUT HERE (should be a csv with no header)
nameOfInputCSVFile = "3Domains.csv"
nameOfExperimentalResultsFile = "data4Optimizer.csv"

## USER SHOULD NOT MODIFY LINES BELOW THIS


expData = pd.read_csv(nameOfInputCSVFile,header=None)
    
#If extra columns get read in, trim them down to just 3
if expData.shape[1] >=4:
    expData = expData.loc[:,1:4]
    
# Name the columsn of the iput data
expData.columns = ["TC", "thr","M", "delM"]


# Calculate Daa from experimental results
expResults = D0calc_MonteCarloErrors(expData)

# Combine the diffusion parameters with the experimental setup (T, thr, M, delM)
# to get a final dataframe that will be passed into the optimizer
diffusionExperimentResults = expData.join(expResults)

# Write dataframe to a .csv file
diffusionExperimentResults.to_csv(nameOfExperimentalResultsFile)

In [27]:
# # Now I'll try the same optimization with the built in scipy global optimizer to see if it does a better job or is faster...
# from scipy import optimize
# bounds = [(70,110),(0,25),(0,25),(0,25),(10**(-10),1),(10**(-10),1)]
# args = (diffusionExperimentResults["TC"].to_numpy(),diffusionExperimentResults["thr"].to_numpy(), diffusionExperimentResults["ln(D/a^2)"].to_numpy(),diffusionExperimentResults["Fi"].to_numpy())
# results = optimize.dual_annealing(objectiveFunction4PythonOptimizer,bounds,args = args)


# fwdModel = forwardModelKinetics(results.x,diffusionExperimentResults)
# plotModelResults(fwdModel,diffusionExperimentResults)

# diffusionExperimentResults
# results

TypeError: sum(): argument 'input' (position 1) must be Tensor, not numpy.ndarray

# Some Notes for Josh 

The inputs to the optimizer have some constraints, and you can see that I've badly enforced them by assigning high misfit values if they aren't satisfied. 

## Model has $2* num Domains +1$ parameters. They are as follows:

#### Ea: Activation energy in kj/mol

#### $\ln(D_0/a^2)$: in (1/s). There is one of these per domain

#### Fraction of total gas (There are $numDom-1$ of these because $\sum_{i=1}^{\#Dom} Frac_i = 1$, so if you know $numDom-1$ of them, you know the value of last Frac).

## The input parameters vector is as follows: [(Ea, lnd0aa1, lnd0aa2,...lnd0aa_x,Frac_1,Frac_2,...Frac_(numDom-1))]


### The constraints on the inputs are as follows:

#### $\ln(D_0/a^2)_1 > ln(D_0/a^2)_2 > ... ln(D_0/a^2)_{numDom}  $

#### $Fracs_1 + Fracs_2 + Fracs3... + Fracs_ndom = 1  $ However, we only stick the first numDom-1 into the optimizer, since the last Frac is determined by the other numDom-1 params

#### Fi_MDD[-1] must = 1


Note: The data we're playing with is a synthetic dataset I created with known values for all of these parameters and it has 3 domains. When solving for real data, we won't know the number of domains, but this is a good way for us to make sure the optimizer is behaving properly.

In [33]:
# I wrote this gross while loop to try to test out the possible values of the model


threshold = .01
num_dim = 6
num_sampToTry = range(3,21)
num_ResampToTry = range(3,20)
misfitVals = np.ones([101,101])*10*11
numIterations4Save = np.ones([100,101])*10*11
durations = np.ones([101,101])*10*11



for i in num_sampToTry:
    j=i
    while j <=100:

        misfit = 10**11
        counter = 0
        print("num_samp = " +str(j))
        print("num_resamp = " +str(i))
        srch = Optimizer(
                objective_fn=objectiveFunction,
                names = ["Ea","LnD0aa1","LnD0aa2","LnD0aa3","Frac1","Frac2"],
                limits=[(70,110),(0,25),(0,25),(0,25),(10**(-10),1),(10**(-10),1)], 
                dataset_path = nameOfExperimentalResultsFile,
                num_samp=8, #number of random samples taken at each iteration
                num_resamp=4, #number of best Voronoi polygons sampled at each iteration-- must be smaller than num_samp
                maximize=False,
                verbose=True
                )
        start_time = time.time()
        didNotconverge = 0
        numIters = 0
        while (misfit >= threshold) and (didNotconverge == 0): # While misfit is below the threshold and we haven't failed to converge
            srch.update(100)
            misfit = srch.sample[0]["result"]
            numIters = numIters+1
            if numIters > 2000:
                didNotconverge = 1
            elif (numIters > 100) and (srch.sample[0]["result"] == srch.sample[75]["result"]):
                didNotConverge = 1
        if didNotconverge != 1: #This means we converged
            misfitVals[i,j] = misfit
            durations[i,j] = (time.time() - start_time)
            numIterations4Save[i,j] = numIters
            j=j+1
  
        
        
#     # Extract the best fitting model with this while loop
#         exitFlag = 0
#         counter = 0
#         while exitFlag == 0:
#             if srch.sample[counter]['iter'] == n_iter-1:
#                 bestFit = srch.sample[counter]
#                 exitFlag = 1
#             else:
#                 counter += 1

#         misfitVals[i-1,j-1] = srch.sample[counter]["result"]

# fwdModel = forwardModelKinetics(srch.sample[counter]['param'],diffusionExperimentResults)
# plotModelResults(fwdModel,diffusionExperimentResults)
# bestFit


np.savetxt("misfitVals.csv", misfitVals,delimiter = ',')
np.savetxt("numiters.csv", numIterations4Save, delimiter = ',')
np.savetxt("durations.csv", durations, delimiter = ',')


num_samp = 3
num_resamp = 3
Optimizer(iteration=1, samples=8, best=1.000000e+10)
Optimizer(iteration=2, samples=16, best=2.348097e+01)
Optimizer(iteration=3, samples=24, best=2.348097e+01)
Optimizer(iteration=4, samples=32, best=2.348097e+01)
Optimizer(iteration=5, samples=40, best=2.348097e+01)
Optimizer(iteration=6, samples=48, best=2.348097e+01)
Optimizer(iteration=7, samples=56, best=1.306059e+00)
Optimizer(iteration=8, samples=64, best=1.306059e+00)
Optimizer(iteration=9, samples=72, best=1.306059e+00)
Optimizer(iteration=10, samples=80, best=1.306059e+00)
Optimizer(iteration=11, samples=88, best=1.306059e+00)
Optimizer(iteration=12, samples=96, best=5.280250e-01)
Optimizer(iteration=13, samples=104, best=5.280250e-01)
Optimizer(iteration=14, samples=112, best=5.280250e-01)
Optimizer(iteration=15, samples=120, best=5.280250e-01)
Optimizer(iteration=16, samples=128, best=5.280250e-01)
Optimizer(iteration=17, samples=136, best=5.280250e-01)
Optimizer(iteration=18, samples=144, best=

Optimizer(iteration=146, samples=1168, best=7.351212e-02)
Optimizer(iteration=147, samples=1176, best=7.351212e-02)
Optimizer(iteration=148, samples=1184, best=7.119973e-02)
Optimizer(iteration=149, samples=1192, best=6.943941e-02)
Optimizer(iteration=150, samples=1200, best=6.943941e-02)
Optimizer(iteration=151, samples=1208, best=6.943941e-02)
Optimizer(iteration=152, samples=1216, best=6.943941e-02)
Optimizer(iteration=153, samples=1224, best=6.943941e-02)
Optimizer(iteration=154, samples=1232, best=6.943941e-02)
Optimizer(iteration=155, samples=1240, best=6.943941e-02)
Optimizer(iteration=156, samples=1248, best=6.943941e-02)
Optimizer(iteration=157, samples=1256, best=6.943941e-02)
Optimizer(iteration=158, samples=1264, best=6.943941e-02)
Optimizer(iteration=159, samples=1272, best=6.943941e-02)
Optimizer(iteration=160, samples=1280, best=6.943941e-02)
Optimizer(iteration=161, samples=1288, best=6.943941e-02)
Optimizer(iteration=162, samples=1296, best=6.943941e-02)
Optimizer(iter

Optimizer(iteration=290, samples=2320, best=5.188871e-02)
Optimizer(iteration=291, samples=2328, best=5.188871e-02)
Optimizer(iteration=292, samples=2336, best=5.188871e-02)
Optimizer(iteration=293, samples=2344, best=5.188871e-02)
Optimizer(iteration=294, samples=2352, best=5.188871e-02)
Optimizer(iteration=295, samples=2360, best=5.188871e-02)
Optimizer(iteration=296, samples=2368, best=5.188871e-02)
Optimizer(iteration=297, samples=2376, best=5.188871e-02)
Optimizer(iteration=298, samples=2384, best=5.188871e-02)
Optimizer(iteration=299, samples=2392, best=5.188871e-02)
Optimizer(iteration=300, samples=2400, best=5.154017e-02)
Optimizer(iteration=301, samples=2408, best=5.154017e-02)
Optimizer(iteration=302, samples=2416, best=5.134153e-02)
Optimizer(iteration=303, samples=2424, best=5.085940e-02)
Optimizer(iteration=304, samples=2432, best=5.085940e-02)
Optimizer(iteration=305, samples=2440, best=5.085940e-02)
Optimizer(iteration=306, samples=2448, best=5.085940e-02)
Optimizer(iter

Optimizer(iteration=432, samples=3456, best=3.826436e-02)
Optimizer(iteration=433, samples=3464, best=3.826436e-02)
Optimizer(iteration=434, samples=3472, best=3.826436e-02)
Optimizer(iteration=435, samples=3480, best=3.826436e-02)
Optimizer(iteration=436, samples=3488, best=3.826436e-02)
Optimizer(iteration=437, samples=3496, best=3.799109e-02)
Optimizer(iteration=438, samples=3504, best=3.799109e-02)
Optimizer(iteration=439, samples=3512, best=3.799109e-02)
Optimizer(iteration=440, samples=3520, best=3.752369e-02)
Optimizer(iteration=441, samples=3528, best=3.752369e-02)
Optimizer(iteration=442, samples=3536, best=3.752369e-02)
Optimizer(iteration=443, samples=3544, best=3.752369e-02)
Optimizer(iteration=444, samples=3552, best=3.752369e-02)
Optimizer(iteration=445, samples=3560, best=3.752369e-02)
Optimizer(iteration=446, samples=3568, best=3.752369e-02)
Optimizer(iteration=447, samples=3576, best=3.752369e-02)
Optimizer(iteration=448, samples=3584, best=3.752369e-02)
Optimizer(iter

KeyboardInterrupt: 